<a href="https://colab.research.google.com/github/arssite/GENAi-Assessment/blob/main/DocumentQAusing_DeepsetRobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 666.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00


In [3]:

!pip install python-docx fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35614 sha256=2321aa0806000941f9a1231e83f8d33b3faf79c2ac280ac3cc5daa

In [4]:
import torch
import gradio as gr
from transformers import pipeline
import fitz  # PyMuPDF for PDF handling
from docx import Document  # For handling .docx files

model_path = ("../Models/models--deepset--roberta-base-squad2/snapshots"
              "/cbf50ba81465d4d8676b8bab348e31835147541b")

question_answer = pipeline("question-answering",
                           model="deepset/roberta-base-squad2")

def read_file_content(file_obj):
    try:
        # Determine the file extension
        file_extension = file_obj.name.split('.')[-1].lower()

        if file_extension == 'txt':
            # Reading text files
            with open(file_obj.name, 'r', encoding='utf-8') as file:
                context = file.read()

        elif file_extension == 'pdf':
            # Reading PDF files using PyMuPDF (fitz)
            with fitz.open(file_obj.name) as pdf:
                context = ""
                for page_num in range(pdf.page_count):
                    page = pdf.load_page(page_num)
                    context += page.get_text()

        elif file_extension == 'docx':
            # Reading Word documents using python-docx
            doc = Document(file_obj.name)
            context = "\n".join([para.text for para in doc.paragraphs])

        else:
            return "Unsupported file format. Please upload a .txt, .pdf, or .docx file."

        return context

    except Exception as e:
        return f"An error occurred: {e}"

def get_answer(file, question):
    context = read_file_content(file)
    if "An error occurred" in context or "Unsupported" in context:
        return context  # Return error message directly if present
    answer = question_answer(question=question, context=context)
    return answer["answer"]

demo = gr.Interface(
    fn=get_answer,
    inputs=[
        gr.File(label="Upload your file"),
        gr.Textbox(label="Input your question", lines=1)
    ],
    outputs=[gr.Textbox(label="Answer text", lines=1)],
    title="@GenAILearniverse Project 5: Document Q & A",
    description="THIS APPLICATION WILL BE USED TO ANSWER QUESTIONS BASED ON CONTEXT PROVIDED."
)

demo.launch()


ModuleNotFoundError: No module named 'frontend'